# EE194 Lab 0: Chisel - Part 2: Sequential Circuits, Collections & Memories
Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE" or `YOUR ACTION NEEDED HERE`.

If you see `???` right below `YOUR CODE HERE`, make sure to remove that after you have implemented your solution (and before you run the code block).

### Import the necessary Chisel dependencies. 
> There will be cells like these in every lab. Make sure you run them before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
interp.configureCompiler(_.settings.processArguments(List("-Wconf:cat=deprecation:s"), true))
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Registers
> We can use registers to store state across multiple cycles. In Chisel, to build registers we use `Reg` as well as `RegNext` and `RegInit`. A TL;DR is provided below, for more, see [here](https://www.chisel-lang.org/chisel3/docs/explanations/sequential-circuits.html).

### Declare a register - `Reg(type)`
* `clock` & `reset` logic is handled implicitly by Chisel
    * For multiple clock domains, see [here](https://www.chisel-lang.org/docs/explanations/multi-clock)
    * For how to implement an async reset, see [here](https://www.chisel-lang.org/docs/explanations/reset)
* Reg is not initialized with a (known) value (aka: it'll be initialized with garbage). If you want a register to be set to a value during reset, use `RegInit(init)`.
* To set a value, use any connect operator (ex: `:=` -- there are others, see [here](https://www.chisel-lang.org/docs/explanations/connectable))

### Set Initial Value - `RegInit(init)`
* Value applied synchronously when reset true

### Attach Input - `RegNext(next)`
* Useful for delaying a signal by a cycle

### Enable - `RegEnable(next, optional: init, en)`
* Write enable for when to update

### Example: Counter (in a few different ways)
Here are 2 examples of a Counter, one using `RegInit` and one ultra-condensed version using `RegEnable` (possible thanks to Chisel abstractions)!

In [ ]:
class MyCounterRegInit(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val enable = Input(Bool())
        val out = Output(UInt()) // Chisel will infer the width here. This is good for simple designs, but as your design gets more complex, you would want to specify the width here... in this case you probably want to write something like: `Output(UInt(log2Ceil(maxVal + 1).W))`
    })

    val count = RegInit(0.U(log2Ceil(maxVal + 1).W)) // the log2Ceil here will ensure `out`'s width is bound.
    val nextVal = Mux(count < maxVal.U, count + 1.U, 0.U)
    count := Mux(io.enable, nextVal, count)
    io.out := count
}
printVerilog(new MyCounterRegInit(15))

In [ ]:
class MyCounterCondensed(maxVal: Int) extends Module{
    val io = IO(new Bundle {
        val en = Input(Bool())
        val out = Output(UInt(log2Ceil(maxVal + 1).W))
    })
    io.out := RegEnable(Mux(io.out < maxVal.U, io.out + 1.U, 0.U), 0.U, io.en)
}
printVerilog(new MyCounterCondensed(15))

### Delay 2
> Fill in the `Delay2` module such that `out` signal is equal to the `in` signal delayed by 2 cycles.

In [ ]:
class Delay2 extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(5.W))
        val out = Output(UInt(5.W))
    })
    
    // YOUR CODE HERE
    ???
}

In [ ]:
def testDelay2: Boolean = {
    test(new Delay2) { dut =>
        // Cycle0
        dut.io.in.poke(5.U)
        dut.io.out.expect(0.U)
        dut.clock.step(1)

        // Cycle1
        dut.io.in.poke(4.U)
        dut.io.out.expect(0.U)
        dut.clock.step(1)

        // Cycle2
        dut.io.in.poke(3.U)
        dut.io.out.expect(5.U)
        dut.clock.step(1)

        // Cycle3
        dut.io.out.expect(4.U)
        dut.clock.step(1)

        // Cycle4
        dut.io.out.expect(3.U)
        dut.clock.step(1)
    }
    true
}
assert(testDelay2)

## State Machines
With our knowledge of registers, we can now track state & construct FSMs.

`ChiselEnum` provides [enumerations](https://www.chisel-lang.org/docs/explanations/chisel-enum) by assigning them `UInt`s 
* Do not confuse this with `Enumeration`, which is a Scala concept

`ChiselEnum` is helpful for implementing FSM states as it allows you to put human-sensical names to distinct values (and not have to worry about what values get assigned to your states).

You might also see `Enum` in older Chisel code -- They function the same, it's just old Chisel that hasn't been updated yet.

In [ ]:
object DemoEnum extends ChiselEnum {
  val nameA, nameB, nameC = Value
  val nameD = Value(5.U) // explicitly assign the value 5 to nameD
}

println(DemoEnum.nameA, DemoEnum.nameB, DemoEnum.nameC, DemoEnum.nameD)

### Example: Raccoon 🦝

![](./images/raccoon_fsm.svg)

The first example uses `when`, `.elsewhen`, `.otherwise` statements.

The second examples uses `switch` statements.

In [ ]:
object RaccAction extends ChiselEnum {
    val hide, wander, rummage, eat = Value
}
class Raccoon extends Module { // using when
    val io = IO(new Bundle {
        val noise = Input(Bool())
        val trash = Input(Bool())
        val food  = Input(Bool())
        val action = Output(RaccAction())
    })
    val state = RegInit(RaccAction.hide)
    when (state === RaccAction.hide) {
        when (!io.noise)     { state := RaccAction.wander }
    } .elsewhen (state === RaccAction.wander) {
        when (io.noise)      { state := RaccAction.hide }
        .elsewhen (io.trash) { state := RaccAction.rummage }
    } .elsewhen (state === RaccAction.rummage) {
        when (io.noise)      { state := RaccAction.hide }
        .elsewhen (io.food)  { state := RaccAction.eat }
    } .elsewhen (state === RaccAction.eat) {
        when (io.noise)      { state := RaccAction.hide }
        .elsewhen (!io.food) { state := RaccAction.wander }
    }
    io.action := state
}
printVerilog(new Raccoon)

In [ ]:
object RaccAction extends ChiselEnum {
    val hide, wander, rummage, eat = Value
}
class Raccoon extends Module { // using switch
    val io = IO(new Bundle {
        val noise = Input(Bool())
        val trash = Input(Bool())
        val food  = Input(Bool())
        val action = Output(RaccAction())
    })
    val state = RegInit(RaccAction.hide)
    switch(state) {
        is (RaccAction.hide) {
            when (!io.noise)     { state := RaccAction.wander }
        }
        is (RaccAction.wander) {
            when (io.noise)      { state := RaccAction.hide }
            .elsewhen (io.trash) { state := RaccAction.rummage }
        }
        is (RaccAction.rummage) {
            when (io.noise)      { state := RaccAction.hide }
            .elsewhen (io.food)  { state := RaccAction.eat }
        }
        is (RaccAction.eat) {
            when (io.noise)      { state := RaccAction.hide }
            .elsewhen (!io.food) { state := RaccAction.wander }
        }
    }
    io.action := state
}
printVerilog(new Raccoon)

In [ ]:
test(new Raccoon()) { r =>
    r.io.noise.poke(1.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(RaccAction.hide)

    r.io.noise.poke(0.B)
    r.clock.step()
    r.io.action.expect(RaccAction.wander)

    r.io.trash.poke(1.B)
    r.clock.step()
    r.io.action.expect(RaccAction.rummage)

    r.io.trash.poke(0.B)
    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(RaccAction.eat)

    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(RaccAction.eat)

    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(RaccAction.wander)

    r.io.noise.poke(1.B)
    r.clock.step()
    r.io.action.expect(RaccAction.hide)
}

### Tapeout FSM

> Write Chisel to implement the following FSM using whatever method you prefer. You may assume each transition is a `Bool` input to your Module.

![](./images/fsm_exec.svg)

In [ ]:
// YOUR CODE HERE
???


In [ ]:
def testTOFSM: Boolean = {
      test(new TapeoutFSM()) { dut =>
            def pulse(sig: chisel3.Bool): Unit = { sig.poke(true.B); dut.clock.step(); sig.poke(false.B); }

            // reset
            dut.reset.poke(true.B); dut.clock.step(); dut.reset.poke(false.B)

            // initial
            dut.io.action.expect(FSMState.arch_design)

            // arch_design: arch_spec -> RTL
            pulse(dut.io.arch_spec)
            dut.io.action.expect(FSMState.RTL)

            // RTL: rtl_sims_failed -> RTL (self-loop)
            pulse(dut.io.rtl_sims_failed)
            dut.io.action.expect(FSMState.RTL)

            // RTL: rtl_pass -> verification
            pulse(dut.io.rtl_pass)
            dut.io.action.expect(FSMState.verification)

            // verification: need_more_coverage -> verification (self-loop)
            pulse(dut.io.need_more_coverage)
            dut.io.action.expect(FSMState.verification)

            // verification: verification_fail -> RTL
            pulse(dut.io.verification_fail)
            dut.io.action.expect(FSMState.RTL)

            // RTL: rtl_pass -> verification
            pulse(dut.io.rtl_pass)
            dut.io.action.expect(FSMState.verification)

            // verification: verification_impossible -> arch_design
            pulse(dut.io.verification_impossible)
            dut.io.action.expect(FSMState.arch_design)

            // arch_design: confusion -> arch_design (explicit self-loop)
            pulse(dut.io.confusion)
            dut.io.action.expect(FSMState.arch_design)
      }
      true
}

assert(testTOFSM)

## Operating on a sequence of components

### `Seq`
In Scala, `Seq` (sequence) is an ordered collection, immutable by default. Index into it using `()`


For context, here is the entire Scala `scala.collection.immutable` library, which `Seq` is a component of.
![](./images/collections-immutable-diagram-213.svg)

Scala Doc: https://www.scala-lang.org/api/2.13.17/scala/collection/immutable/Seq.html

In [ ]:
val l = Seq(1,2,3)
l(1)
l.isEmpty
l.nonEmpty
l.length
Seq.fill(3)(8)
l.getClass

### `Range`
* `Range` is a special `IndexdSeq` that represents the set of integer values in the defined range -- typically `[start;end)`
* Syntax: `start until end` (exclusive)
* Syntax: `start to end` (inclusive)
* Can provide a custom "increment by" value (default is `1`) via the keyword `by`

Scala Doc Reference: https://www.scala-lang.org/api/2.13.17/scala/collection/immutable/Range.html

In [ ]:
0 until 4
0 to 3
0 to 4 by 2
3 to 0 by -1

### Scala Loops
* Can iterate over a `iterable`, but generally this is either a `range`.
* Can do fancy list comprehensions... Docs: https://docs.scala-lang.org/tour/for-comprehensions.html
* Can use in a Scala Test!
* The syntax is pretty self-explanatory, Google it if you need a reference.

> To familiarize ourselves with looping in Scala, we will attempt a variant of the game [_Fizz buzz_](https://en.wikipedia.org/wiki/Fizz_buzz):

* _*Count*_ from 1 until `max` (exclusive)
* Maintain three sums: `fizz`, `buzz`, and `fizzbuzz`
* If the current count is divisible by ...
    * _15_ $\rightarrow$ add count to `fizzbuzz`
    * _5_ $\rightarrow$ add count to `buzz`
    * _3_ $\rightarrow$ add count to `fizz`
    * Add to the sums with the precedence rules given, so for example, if the count is _15_, only add to `fizzbuzz`.

In [ ]:
def fizzbuzz(max: Int): (Int, Int, Int) = {
    var (fizz, buzz, fizzbuzz) = (0, 0, 0)

    // YOUR CODE HERE
    ???
}

fizzbuzz(20)

In [ ]:
assert(fizzbuzz(2) == (0, 0, 0))
assert(fizzbuzz(20) == (48, 15, 15))
assert(fizzbuzz(35) == (153, 60, 45))

## Using Scala `for` with Chisel
* Scala `for` in the context of Chisel very much behaves like Verilog's `for` loop -- it is used to replicate or iterate over a set of hardware logic.

> Complete the Chisel code below to implement a Delay Line -- Logic that outputs a specified input after a certain number of clock ticks. Once your implementation passes, we'll show you an implementation that uses `var` for your reference. We've almost exclusively used `val` up to this point, but we want to show you that judicious use of `var` can help!

![](./images/delayN.svg)

In [ ]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n > 0)
    val regs = /* YOUR CODE HERE - HINT: We need a *collection* of registers so data can be passed through them -- what creates that? - Scroll up! */ 

    regs(0) := io.in
    
    // YOUR CODE HERE
    ???
    
    io.out := /* YOUR CODE HERE */
}

In [ ]:
import scala.io.Source
import almond.display.Markdown

def testDelayNCycles: Boolean = {
    test(new DelayNCycles(5)) { c => 
        c.io.in.poke(true.B)
        c.clock.step(1)
        c.io.in.poke(false.B)
        c.clock.step(4)
        c.io.out.expect(true.B)
        c.clock.step(1)
        c.io.out.expect(false.B)

        c.io.in.poke(false.B)
        c.clock.step(1)
        c.io.in.poke(true.B)
        c.clock.step(4)
        c.io.out.expect(false.B)
        c.clock.step(1)
        c.io.out.expect(true.B)
    }
    true
}
assert(testDelayNCycles)

val DelayNCyclesVar = Source.fromFile("./DelayNCyclesVar.scala").mkString
Markdown(s"```scala\n$DelayNCyclesVar\n```")

## Using Chisel Vecs to Implement a Lookup Table
> Vecs allow us to dynamically index into a Chisel collection during operation in the generated hardware.

> In the problem, we use a Vec to implement a _lookup table_, which is a read-only memory (ROM) that holds precomputed results. The `SquareTable` module we provide below produces the result of squaring its input. However, the generated hardware contains only the lookup table because the multiplication is done at generation time. As a first implementation, we hardcode the IO widths and table size.

Note that you can use Scala `for`s in Chisel test modules as well. The following `testSquareTable` exhaustively tests all of `SquareTable`'s inputs (there are 32).

In [ ]:
class SquareTable extends Module {
    val io = IO(new Bundle {
        val x = Input(UInt(5.W))
        val xSquared = Output(UInt(10.W))
    })
    // 0.U, 1.U, ..., 31.U
    val romData = Seq.tabulate(32)(i => (i*i).U(10.W))
    show(s"romData: $romData")

    val ROM: Vec[UInt] = VecInit(romData)
    io.xSquared := ROM(io.x)
}

In [ ]:
def testSquareTable: Boolean = {
    test(new SquareTable) { dut =>
        for (i <- 0 until 32) {
            dut.io.x.poke(i.U)
            dut.io.xSquared.expect((i*i).U)
        }
    }
    true
}

In [ ]:
assert(testSquareTable)

### Parameterizing SquareTable
In this problem, we will revise the module `SquareTable` from the previous problem to make the number of entries parameterized. Write a `SquareTable2` module below that takes a Scala Int `nEntries` as a parameter. `SquareTable2` will generate a lookup table (read-only UInts stored as a `Vec`) of nEntries entries. The output `xSquared` should be the result of squaring the input `x`. The table will support inputs from 0 to nEntries-1 (inclusive). Ensure the widths are correct, namely that x is wide enough to handle all inputs (hint: consider log2Ceil), and xSquared is sufficiently wide to not loose any data. `SquareTable2` behaves the same as `SquareTable`, except its size is a configurable parameter.

In [ ]:
class SquareTable2(nEntries: Int) extends Module {
    val io = IO(new Bundle {
        // YOUR CODE HERE
        val x = ???
        val xSquared = ???
    })
    // 0.U, 1.U, ..., 31.U
    
    // YOUR CODE HERE
    ???

    show(s"romData: $romData")

    val ROM: Vec[UInt] = VecInit(romData)
    io.xSquared := ROM(io.x)
}

In [ ]:
def testSquareTable2: Boolean = {
    for (nEntries <- Seq(4, 5, 10, 30)) {
        test(new SquareTable2(nEntries)) { dut =>
            for (i <- 0 until nEntries) {
                dut.io.x.poke(i.U)
                dut.io.xSquared.expect((i*i).U)
            }
        }
    }
    true
}
assert(testSquareTable2)

## Histogram
Let's put Regs and Vecs together to build a `Histogram` generator. The generated hardware will count how often it sees each input. It will take a parameter `n`, and internally it will use `n` registers to count how often it has seen inputs (from 0 to n-1). Each cycle, the `x` input chooses which internal register to increment. The `out` output should provide the new total value that will be written into the register the next cycle. In other words, `out` immediately shows new value of the count associated with `x`.

In [ ]:
class Histogram(n: Int) extends Module {
    val io = IO(new Bundle {
        val x = Input(UInt(log2Ceil(n).W))
        val out = Output(UInt(5.W))
    })
    // YOUR CODE HERE
    ???
}

In [ ]:
def testHistogram: Boolean = {
    test(new Histogram(3)) { dut =>
        // Cycle0
        dut.io.x.poke(1.U)
        dut.io.out.expect(1.U)
        dut.clock.step(1)

        // Cycle1
        dut.io.x.poke(1.U)
        dut.io.out.expect(2.U)
        dut.clock.step(1)

        dut.io.x.poke(0.U)
        dut.io.out.expect(1.U)
        dut.clock.step(1)

        // Cycle 3
        dut.io.x.poke(2.U)
        dut.io.out.expect(1.U)
    }
    true
}
assert(testHistogram)

## Chisel Memories: `Mem` & `SyncReadMem`
* This is what you use to implement Read-Write memories
* Chisel (and later, in conjunction with Chipyard) will handle mapping them down to FPGA memories, ASIC memories.
* Supports masking

### `SyncReadMem`: sequential/synchronous-read, sequential/synchronous-write
* Use `SyncReadMem` (as opposed to `Mem`) to hint to Chisel to map your memory down to an SRAM

### `Mem`: combinational/asynchronous-read, sequential/synchronous-write
* These will likely get mapped down to giant register banks as opposed to technology SRAMs or BRAMs

Docs: https://www.chisel-lang.org/docs/explanations/memories

### Register File (3 ways)
* We use `Mem` here because we want a giant register bank for the RegFile. For examples of `SyncReadMem`, check the documentation link above.
* First implementation uses explicit ports (2R, 1W)
* Second implementation uses implicit ports (aka: let Chisel figure out which is a read port and which is a write port -- it'll again be a 2R, 1W)
* Third implementation parametrizes the read port (so we have `N` read, 1W ports)

In [ ]:
class RegFile extends Module {
    val io = IO(new Bundle {
        val r0addr = Input(UInt(5.W))
        val r1addr = Input(UInt(5.W))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val r0out =  Output(UInt(64.W))
        val r1out =  Output(UInt(64.W))
    })
    val regs = Mem(32, UInt(64.W))
    io.r0out := regs.read(io.r0addr)
    io.r1out := regs.read(io.r1addr)
    when (io.w0en) {
        regs.write(io.w0addr,io.w0data)
    }
}
printVerilog(new RegFile)

In [ ]:
class RegFile() extends Module {
    val io = IO(new Bundle {
        val r0addr = Input(UInt(5.W))
        val r1addr = Input(UInt(5.W))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val r0out =  Output(UInt(64.W))
        val r1out =  Output(UInt(64.W))
    })
    val regs = Mem(32, UInt(64.W))
    io.r0out := regs(io.r0addr)
    io.r1out := regs(io.r1addr)
    when(io.w0en) {
        regs(io.w0addr) := io.w0data
    }
}
printVerilog(new RegFile)

In [ ]:
class RegFile(nRead: Int) extends Module {
    val io = IO(new Bundle {
        val raddr  = Input(Vec(nRead, UInt(5.W)))
        val w0addr = Input(UInt(5.W))
        val w0en =   Input(Bool())
        val w0data = Input(UInt(64.W))
        val rout = Output(Vec(nRead, UInt(64.W)))
    })
    val regs = Mem(32, UInt(64.W))
    for (i <- 0 until nRead)
        io.rout(i) := regs(io.raddr(i))
    when(io.w0en) {
        regs(io.w0addr) := io.w0data
    }
}
printVerilog(new RegFile(4))

## Summary - When to Use Scala or Chisel Collection?
* You will want to use a Scala collection (probably `Seq`) most often
    * In a generator, want to instantiate N things
    * You may need to address which thing you want, but you are accessing it during hardware generation time
* You will need Chisel collections (`Vec`, `Mem`) when ...
    * You want the generated hardware to dynamically address components (put muxes into actual hardware)
    * You definitely want a memory (e.g. `SyncReadMem`)